### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [3]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [4]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
tfidfvect = TfidfVectorizer()

In [5]:
# en el atributo `data` accedemos al texto
print(newsgroups_train.data[0])

I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.


In [6]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [7]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'Cantidad de documentos: {X_train.shape[0]}')
print(f'Tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
Cantidad de documentos: 11314
Tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [8]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [39]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [40]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [41]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [42]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [43]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [44]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [45]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([ 4811,  6635,  4253, ...,  4703, 10870,  4333])

In [46]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [47]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [48]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [49]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [50]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [51]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.


## Primer punto

Para el primer punto creo una función para generar las muestras y buscar los documentos mas similares.

In [52]:

def encontrar_mas_cercanos(X_train, y_train, newsgroups_train):

  # Seleccionar un documento al azar
  idx = np.random.randint(0, X_train.shape[0])
  print(f"-------------- Indice del documento original: {idx} --------------")
  # Calcular la similaridad coseno con todos los documentos
  cossim = cosine_similarity(X_train[idx], X_train)[0]

  # Encontrar los 5 documentos más similares
  mostsim = np.argsort(cossim)[::-1][1:6]

  # Imprimir la categoría del documento original
  print(f"Documento original: categoría {newsgroups_train.target_names[y_train[idx]]}")
  print(newsgroups_train.data[idx])

  # Imprimir las categorías y el contenido de los 5 documentos más similares
  for i in mostsim:

    print(f"-------------- Indice del documento relacionado: {i} --------------" )
    print(f"\nDocumento similar: categoría {newsgroups_train.target_names[y_train[i]]}")
    print(newsgroups_train.data[i])




In [53]:
encontrar_mas_cercanos(X_train, y_train, newsgroups_train)

-------------- Indice del documento original: 2080 --------------
Documento original: categoría talk.religion.misc
Well, that would depend on how much we wanted the US and how much
we wanted the $1, wouldn't it?
-Ekr

-------------- Indice del documento relacionado: 10632 --------------

Documento similar: categoría rec.autos
I just wanted to know:
-------------- Indice del documento relacionado: 432 --------------

Documento similar: categoría sci.space
The following are my thoughts on a meeting that I, Hugh Kelso, and Bob Lilly
had with an aide of Sen. Patty Murrays.  We were there to discuss SSTO, and
commercial space.  This is how it went...



After receiving a packet containing a presentation on the benifits of SSTO,
I called and tried to schedule a meeting with our local Senator (D) Patty
Murray, Washington State.  I started asking for an hour, and when I heard
the gasp on the end of the phone, I quickly backed off to 1/2 an hour.
Later in that conversation, I learned that a sta

**Documento 5297**

 Solicita información sobre FAQ, los documentos relacionados 3 hablan explicitamente sobre FAQs y de los otros dos se pide información usando el mismo modismo "Could some kind soul".

In [23]:
encontrar_mas_cercanos(X_train, y_train, newsgroups_train)

-------------- Indice del documento original: 9000 --------------
Documento original: categoría talk.politics.guns

: I have seen these numbers quoted before, and I have seen very specific
: refutation of them quoted as well.  If someone will be so kind as to
: email the relevant information, I will write a letter to the editor of
: the Co. Daily (which might get published) and send a copy to USN&WR as
: well.

Thanks to all who responded.  The letter has been written (making liberal
use of info provided by various net.folks) and handed to the paper.  I'll
post if it gets into the paper!

--Dan

--
  DoD #202 / loki@acca.nmsu.edu / liberty or death / taylordf@ucsu.colorado.edu 
                 Send me something even YOU can't read...
-----BEGIN PGP PUBLIC KEY BLOCK-----
Version: 2.1
-------------- Indice del documento relacionado: 2057 --------------

Documento similar: categoría talk.politics.guns
The following is quoted from the tail end of a (rather condescending)
article about Pax

**Documento 5297**

El documento original tiene una postura bastante conservadora sobre la esclavitud. Los dos documentos mas cercanos, 9966 y 6234, son relacionado a la esclavitud como era de esperarse, pero de ahí en adelante se ve una relación subyacente.

Los dos documentos que siguen, 913 y 2392, son declaraciones antisemitas por lo tanto se puede ver que aunque no este hablando del tema se conserva de alguna forma la idelogía de los autores.

El documento que más me intereso fue el último, 6880, porque es una defensa de la ocupación británica de las islas malvinas. Me sorprendio la similitud que encontró para sustentar que el colonialismo es extensión del esclavismo.

In [24]:

encontrar_mas_cercanos(X_train, y_train, newsgroups_train)

-------------- Indice del documento original: 10269 --------------
Documento original: categoría comp.sys.ibm.pc.hardware

: How hot should the CPU in a 486-33 DX machine be?

: Currently it gets so hot that I can not hold a finger on it for more than
: 0.5 s. 

I seem to recall that 486s run somewhere close to the boiling point of water.
Anyone have an exact temperature?

Anyway, putting a CPU fan/heat sink on it won't hurt and could help. Depends
on how paranoid you are...
-------------- Indice del documento relacionado: 2716 --------------

Documento similar: categoría comp.sys.ibm.pc.hardware
How hot should the CPU in a 486-33 DX machine be?

Currently it gets so hot that I can not hold a finger on it for more than
0.5 s. 

I keep a big fan blowing on it, but am considering using a heat sink.

Any advice?


-------------- Indice del documento relacionado: 11311 --------------

Documento similar: categoría comp.sys.ibm.pc.hardware
I just installed a DX2-66 CPU in a clone motherboard

**Documento 5551**
Tanto el documento original como  todos los relacionado hablando sobre sistemas operativos, principalmente Windows. Tiene sentido que todos caigan dentro de la categoría "comp.os.ms-windows.misc"

In [25]:
encontrar_mas_cercanos(X_train, y_train, newsgroups_train)

-------------- Indice del documento original: 2161 --------------
Documento original: categoría rec.motorcycles

	Don't just nab it, POUNCE on it.  These are fairly rare bikes, and
they are MORE than adequate for putting a big brown stripe in your shorts.
Does a 50mph power wheelie appeal to you?  I thought it would...

	Only really bad things:  the stock clutch isn't up to the task.
Barnett can take care of this.  The back tire wears quickly (gee, wonder why?),
and the induction system is a bear to work on.

Later,
-------------- Indice del documento relacionado: 2554 --------------

Documento similar: categoría rec.autos


*nnnnnnnng* Thank you for playing, I cannot agree with this.  I believed
this and to put it nicely, it was a piece of junk!

I loved this car, I babied it, I pampered it, and after 2 years, it just
couldn't stay together, I would say that not everyone will have the
problems that I had, but know this, it's not just the car, it is the
ability to get the car fixed, wh

**Documento 10219**

El documento principal habla sobre la posibilidad de alunizar con un agüjero en la pared de una nave espacial. El documento más cercano, 7425, habla sobre agujeros pero en un auto, supongo que es una estimación correcta porque pero menos significativa que el resto de los documentos cercanos que hablan sobre viajes en el espacio.  


In [26]:
encontrar_mas_cercanos(X_train, y_train, newsgroups_train)

-------------- Indice del documento original: 5704 --------------
Documento original: categoría comp.graphics
I recently got a file describing a library of rendering routines 
called SIPP (SImple Polygon Processor).  Could anyone tell me where I can 
FTP the source code and which is the newest version around?
        Also, I've never used Renderman so I was wondering if Renderman 
is like SIPP?  ie. a library of rendering routines which one uses to make 
a program that creates the image...

                                        Thanks,  Joe Tham

-------------- Indice del documento relacionado: 4621 --------------

Documento similar: categoría comp.graphics
Does anyone know of a site where I could ftp some RenderMan shaders?
Or of a newsgroup which has discussion or information about RenderMan?  I'm
new to the RenderMan (Mac) family, and I'd like to get as much info I can
lay my hands on.  Thanks!

        Andy Bates.


----------------------------------------------------------------

**Documento  7648**

Tanto el documento original como todos los cercanos hablan sobre el PowerBook Duo, un modelo de Apple. Los dos documentos mas cercanos comparte tambien la especificidad sobre el problema en si con ese modelo el "Duo Dock", supongo que es un tipo de entrada que inventaron para el modelo y salió mal.



## Segundo punto: clasificación de Bayes

In [24]:
import pandas as pd

In [54]:

vectorizers = [
    CountVectorizer(),
    CountVectorizer(ngram_range=(1, 2)),
    TfidfVectorizer(),
    TfidfVectorizer(ngram_range=(1, 2)),
]

models = [
    MultinomialNB(),
    MultinomialNB(alpha=0.1),
    ComplementNB(),
    ComplementNB(alpha=0.1),
]


# Create an empty list to store the results
classification_results = []

for vectorizer in vectorizers:
    X_train = vectorizer.fit_transform(newsgroups_train.data)
    X_test = vectorizer.transform(newsgroups_test.data)
    for model in models:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        f1 = f1_score(y_test, y_pred, average='macro')
        classification_results.append([str(vectorizer), str(model), f1])

# Create a DataFrame from the results
df = pd.DataFrame(classification_results, columns=['Vectorizer', 'Model', 'F1-score'])

print("\nClassification Results:")
df



Classification Results:


,Vectorizer,Model,F1-score
0,CountVectorizer(),MultinomialNB(),0.512148
1,CountVectorizer(),MultinomialNB(alpha=0.1),0.622667
2,CountVectorizer(),ComplementNB(),0.637427
3,CountVectorizer(),ComplementNB(alpha=0.1),0.638557
4,"CountVectorizer(ngram_range=(1, 2))",MultinomialNB(),0.399548
5,"CountVectorizer(ngram_range=(1, 2))",MultinomialNB(alpha=0.1),0.604131
6,"CountVectorizer(ngram_range=(1, 2))",ComplementNB(),0.658783
7,"CountVectorizer(ngram_range=(1, 2))",ComplementNB(alpha=0.1),0.660989
8,TfidfVectorizer(),MultinomialNB(),0.585435
9,TfidfVectorizer(),MultinomialNB(alpha=0.1),0.656451


Según la puntuación F1, el modelo con mejor rendimiento en este conjunto de experimentos parece ser el que utiliza TfidfVectorizer(ngram_range=(1, 2)) con ComplementNB(alpha=0.1), logrando una puntuación F1 de 0.705837. Esto sugiere que considerar tanto unigramas como bigramas con ponderación TF-IDF y usar ComplementNB con suavizado es una buena opción para este corpus.

## Tercer punto



In [10]:
# Transponer la matriz documento-término
X_train_transposed = X_train.transpose()

# Seleccionar 5 palabras para estudiar su similaridad
words = ['law', 'window', 'board', "ship", 'news']
word_indices = [tfidfvect.vocabulary_[word] for word in words if word in tfidfvect.vocabulary_]


idx2word = {v: k for k, v in tfidfvect.vocabulary_.items()}

# Calcular la similaridad coseno entre las palabras seleccionadas y todas las demás
word_similarities = cosine_similarity(X_train_transposed[word_indices], X_train_transposed)

# Encontrar las 5 palabras más similares a cada una de las palabras elegidas
for i, word_index in enumerate(word_indices):
    word = idx2word[word_index]
    ario
    similar_word_indices = np.argsort(word_similarities[i])[::-1][1:6]
    print(f"Palabras más similares a '{word}':")
    for j in similar_word_indices:

        if j in idx2word:
            # Obtener la palabra similar a partir del índice (ahora en todo el vocabulario)
            similar_word = idx2word[j]
            print(f"- {similar_word}")
    print()

Palabras más similares a 'law':
- enforcement
- deemphasizing
- uncompromising
- herbs
- convicting

Palabras más similares a 'window':
- decoration
- manager
- xmapwindow
- xcreatewindow
- xquerytree

Palabras más similares a 'board':
- autodoubler
- stinky
- diskdoubler
- recompress
- stac

Palabras más similares a 'ship':
- fruited
- uncrated
- wathcing
- 1541b
- prep

Palabras más similares a 'news':
- buggy
- reader
- pageview
- workarounds
- ghostview



**Law**
Tiene una buena representación por estar cercano a otros terminos judiciales.


**Window**
Esta bueno que haga referencia a el objeto real y al concepto en la jerga.


**Board**
Tal vez no tiene una presencia tan significativa como los terminos anteriores porque no entiendo del todo la similitud

**Ship**
Similar a Board, pensaba que lo ubicaría cerca de palabras asociadas al espacio

**News**
pense que lo asociaria a palabras relacionadas a las noticias o el periodismo pero la vinculación no será tan evidente desde el puntod de vista estadistico.